In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub

from keras_preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [3]:
num_of_gpu = len(tf.config.experimental.list_physical_devices('GPU'))

train_on_gpu = num_of_gpu > 0 

In [4]:
root_dir = 'drive/My Drive/Lab' 
# if train_on_gpu else '.'

data_dir = os.path.join(root_dir, 'data/')
output_dir = os.path.join(root_dir, 'output/')

csv_dir = os.path.join(data_dir, 'csv/')
images_dir = os.path.join(data_dir, 'images/')

training_file = os.path.join(csv_dir, 'train_data.csv')

testing_file = os.path.join(csv_dir, 'test_data.csv')

In [5]:
training_df = pd.read_csv(training_file, index_col=0)
testing_df = pd.read_csv(testing_file, index_col=0)

In [6]:
training_df['pneumonia'] = training_df['pneumonia'].astype(str)
testing_df['pneumonia'] = testing_df['pneumonia'].astype(str)

In [7]:
di = {'0': "NORMAL", '1': "PNEUMONIA"}

training_df['pneumonia'].replace(di, inplace=True)
testing_df['pneumonia'].replace(di, inplace=True)

In [8]:
batch_size = 64
image_size = (224, 224)

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,  
    horizontal_flip = True, 
    vertical_flip = False, 
    height_shift_range= 0.1, 
    width_shift_range=0.1, 
    rotation_range=30,
    shear_range = 0.1,
    zoom_range=0.2,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255.)

In [10]:
train_generator=train_datagen.flow_from_dataframe(
    dataframe=training_df,
    directory=images_dir,
    x_col="image_index",
    y_col='pneumonia', 
    batch_size=batch_size,
    seed=42,
    shuffle=True,
    class_mode='categorical',
    target_size=image_size,
    subset='training',
    color_mode='rgb', 
    classes=['NORMAL', 'PNEUMONIA']
)

valid_generator=train_datagen.flow_from_dataframe(
    dataframe=training_df,
    directory=images_dir,
    x_col="image_index",
    y_col='pneumonia', 
    batch_size=batch_size,
    seed=42,
    shuffle=False,
    class_mode='categorical',
    target_size=image_size,
    subset='validation',
    color_mode='rgb', 
    classes=['NORMAL', 'PNEUMONIA']
)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=testing_df,
    directory=images_dir,
    x_col="image_index",
    y_col='pneumonia',
    batch_size=batch_size,
    seed=42,
    shuffle=False,
    class_mode='categorical',
    target_size=image_size,
    color_mode='rgb', 
    classes=['NORMAL', 'PNEUMONIA']
)

Found 3588 validated image filenames belonging to 2 classes.
Found 897 validated image filenames belonging to 2 classes.
Found 1121 validated image filenames belonging to 2 classes.


In [11]:
num_classes = 2

In [12]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

IMG_WIDTH, IMG_HEIGHT = image_size

feature_extractor = hub.KerasLayer(URL, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

feature_extractor.trainable = False

In [13]:
model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1280, activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(540, activation = 'relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1280)              1639680   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 540)               691740    
_________________________________________________________________
dropout_2 (Dropout)          (None, 540)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1

In [14]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
loss, accuracy = model.evaluate(test_generator)

print('\nLoss before training: {:,.3f}'.format(loss))
print('Accuracy before training: {:.3%}'.format(accuracy))

18/18 [==============================] - 24s 1s/step - loss: 0.7704 - accuracy: 0.4710

Loss before training: 0.770
Accuracy before training: 47.101%


In [ ]:
EPOCHS = 500

model_file = os.path.join(output_dir, 'model.h5')

early_stopping = tf.keras.callbacks.EarlyStopping(
                    monitor='val_accuracy', 
                    patience=10
                 )

save_best = tf.keras.callbacks.ModelCheckpoint(
                model_file, 
                monitor='val_accuracy', 
                save_best_only=True
            )

history = model.fit(
    train_generator, 
    epochs = EPOCHS, 
    validation_data=valid_generator, 
    callbacks=[early_stopping, save_best]
)

Epoch 1/500
57/57 [==============================] - 146s 3s/step - loss: 0.9723 - accuracy: 0.5329 - val_loss: 0.6641 - val_accuracy: 0.6187
Epoch 2/500
57/57 [==============================] - 143s 3s/step - loss: 0.7068 - accuracy: 0.5649 - val_loss: 0.6690 - val_accuracy: 0.5797
Epoch 3/500
57/57 [==============================] - 142s 2s/step - loss: 0.6863 - accuracy: 0.5864 - val_loss: 0.6648 - val_accuracy: 0.5964
Epoch 4/500
57/57 [==============================] - 144s 3s/step - loss: 0.6814 - accuracy: 0.5842 - val_loss: 0.6570 - val_accuracy: 0.6288
Epoch 5/500
57/57 [==============================] - 143s 3s/step - loss: 0.6747 - accuracy: 0.5906 - val_loss: 0.6581 - val_accuracy: 0.6221
Epoch 6/500
57/57 [==============================] - 143s 3s/step - loss: 0.6721 - accuracy: 0.5917 - val_loss: 0.6500 - val_accuracy: 0.6276
Epoch 7/500
47/57 [=======================>......] - ETA: 19s - loss: 0.6655 - accuracy: 0.6130

In [ ]:
loss, accuracy = model.evaluate(test_generator)

print('\nLoss after training: {:,.3f}'.format(loss))
print('Accuracy after training: {:.3%}'.format(accuracy))

In [ ]:
final_model_file = os.path.join(output_dir, 'final-model.h5')

model.save(final_model_file)

In [ ]:
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

training_loss = history.history['loss']
validation_loss = history.history['val_loss']

epochs_range=range(len(training_accuracy))

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, training_accuracy, label='Training Accuracy')
plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, training_loss, label='Training Loss')
plt.plot(epochs_range, validation_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()